In [1]:
from psycopg import connect
from biosearch_core.db.model import ConnectionParams, params_from_env
from biosearch_core.bilava import process_figures

env_file = "/home/jtt/kubernetes/envs/.env_training"
conn_params = params_from_env(env_file)

parquets_dir = "/media/cumulus/curation_data/modality_classifiers_production/data/cord19"

classifiers = {
    # "gel": {
    #     "longname": "gel",
    #     "shortname": "exp.gel",
    #     "model": "/media/cumulus/curation_data/modality_classifiers_production/models/cord19/gel/efficientnet-b1_gel_0.pt"
    # },
    "higher-modality": {
        "longname": "higher-modality",
        "shortname": "",
        "model": "/media/cumulus/curation_data/modality_classifiers_production/models/cord19/higher-modality/efficientnet-b1_higher-modality_0.pt"    
    },
    # "experimental": {
    #     "longname": "experimental",
    #     "shortname": "exp",
    #     "model": "/media/cumulus/curation_data/modality_classifiers_production/models/cord19/experimental/efficientnet-b1_experimental_0.pt"    
    # },
    # "experimental": {
    #     "longname": "graphics",
    #     "shortname": "gra",
    #     "model": "/media/cumulus/curation_data/modality_classifiers_production/models/cord19/graphics/efficientnet-b1_graphics_0.pt"    
    # },
    # "microscopy": {
    #     "longname": "microscopy",
    #     "shortname": "mic",
    #     "model": "/media/cumulus/curation_data/modality_classifiers_production/models/cord19/microscopy/efficientnet-b0_microscopy_0.pt"    
    # },
    # "electron": {
    #     "longname": "electron",
    #     "shortname": "mic.ele",
    #     "model": "/media/cumulus/curation_data/modality_classifiers_production/models/cord19/electron/efficientnet-b1_electron_0.pt"   
    # },
    # "molecular": {
    #     "longname": "molecular",
    #     "shortname": "mol",
    #     "model": "/media/cumulus/curation_data/modality_classifiers_production/models/cord19/molecular/efficientnet-b1_molecular_0.pt"   
    # },
    # "radiology": {
    #     "longname": "radiology",
    #     "shortname": "rad",
    #     "model": "/media/cumulus/curation_data/modality_classifiers_production/models/cord19/radiology/efficientnet-b0_radiology_0.pt"   
    # },
    # "radiology": {
    #     "longname": "photography",
    #     "shortname": "pho",
    #     "model": "/media/cumulus/curation_data/modality_classifiers_production/models/cord19/photography/resnet34_photography_0.pt"   
    # },
}

schemas_2_base_img_dir = {
    "training": "/media/cumulus/curation_data",
    "cord19": "/media/cumulus/biosearch/cord19/to_predict"
}

/home/jtt/repos/bio-search/content-onboarding/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import time

def onboard_bilava_data_for_classifier(classifier, training_parques_dir, schemas_2_base_img_dir):
    img_path_2_split_set = process_figures.fetch_split_set(classifier["longname"], training_parques_dir)

    with connect(conninfo=conn_params.conninfo(), autocommit=False) as conn:
      df_db_figures = process_figures.fetch_from_db(conn, classifier["shortname"], ["training", "cord19"])
    
    df_db_figures["split_set"] = df_db_figures.apply(lambda x: img_path_2_split_set[x.uri]["split_set"] if x.uri in img_path_2_split_set else "UNL", axis=1)
    # remove duplicated uri, most likely from training data
    df_db_figures = df_db_figures.set_index("uri")
    df_db_figures = df_db_figures[~df_db_figures.index.duplicated()]
    df_db_figures = df_db_figures.reset_index()

    #return df_db_figures
    df_processed = process_figures.calc_features_and_dim_values(classifier["model"], df_db_figures, schemas_2_base_img_dir)    
    return df_processed

for key in classifiers.keys():
    tic = time.time()
    classifier = classifiers[key]
    print(classifier['longname'])
    
    try:      
      df = onboard_bilava_data_for_classifier(classifier, parquets_dir, schemas_2_base_img_dir)
      df = df.astype({ 'id': int, 'status': int, 'width': int, 'height': int})
      df.to_parquet(f"./processed_{classifier['longname']}.parquet")
      toc = time.time() - tic
      print(toc)
    except Exception as e:
       print(e)

higher-modality


/home/jtt/repos/bio-search/content-onboarding/biosearch_core/bilava/process_figures.py:163: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_schema = sqlio.read_sql_query(query, conn)
/home/jtt/repos/bio-search/content-onboarding/biosearch_core/bilava/process_figures.py:163: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_schema = sqlio.read_sql_query(query, conn)


extracting features


/home/jtt/repos/bio-search/content-onboarding/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jtt/repos/bio-search/content-onboarding/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/jtt/repos/bio-search/content-onboarding/.venv/lib/python3.10/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (110395480 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/jtt/repos/bio-search/content

predicting labels


/home/jtt/repos/bio-search/content-onboarding/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jtt/repos/bio-search/content-onboarding/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/jtt/repos/bio-search/content-onboarding/.venv/lib/python3.10/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (110395480 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/jtt/repos/bio-search/content

calculating active learning metrics
projecting PCA
devices available:  1
running pca on gpu
std::bad_alloc: out_of_memory: CUDA error at: /project/python/_skbuild/linux-x86_64-3.10/cmake-build/_deps/rmm-src/include/rmm/mr/device/cuda_memory_resource.hpp:70: cudaErrorMemoryAllocation out of memory
